In [1]:
import math
import numpy as np
import pandas as pd
import scipy.integrate as integrate
from scipy.stats import norm
from scipy.stats import t
from scipy.stats import invgamma
from scipy.stats import chi2

S0 = 196
r = 0.05
teta = 0.01433
sigma = 0.89

n = 1000

In [2]:
def calculate_parameters():
    historical_data = pd.read_csv("data_ALL/historical_data.csv")
    log_returns = np.log(historical_data['Last']/historical_data['Last'].shift(1))
    log_returns.dropna(inplace=True)
    nu, mu, delta = t.fit(log_returns)
    delta = math.sqrt(delta*252)
    return nu, mu, delta

nu, mu, delta = calculate_parameters()
print("nu = " + str(nu))
print("mu = " + str(mu))
print("delta = " + str(delta))

nu = 12.122281271175913
mu = -0.001771446205980524
delta = 3.123328498823122


In [3]:
def d1_cdf(Y, S, K, sigma, r, t):
    d1 = (math.log(S/K) + r*Y + 0.5*sigma*sigma*t)/(sigma*math.sqrt(t))
    return norm.cdf(d1)

In [4]:
def d2_cdf(Y, S, K, sigma, r, t):
    d2 = (math.log(S/K) + r*Y - 0.5*sigma*sigma*t)/(sigma*math.sqrt(t))
    return norm.cdf(d2)

In [5]:
def fty(Y, t, nu, delta):
    a = (delta*delta)/2
    b = nu/2
    E = (delta*delta)/(nu-2)
    x = (t - E*(math.sqrt(Y) - Y)) / math.sqrt(Y)
    if x <= 0:
        return 0
    else:
        return (math.pow(a,b) / math.gamma(b)) * (x**(-b-1)) * (math.exp(-(a/x)))

In [6]:
def option_price(Y, K, S, d1_cdf, d2_cdf, r, nu, delta, sigma):
    def expr(t):
        d1_t = d1_cdf(Y, S, K, sigma, r, t)
        d2_t = d2_cdf(Y, S, K, sigma, r, t)
        fty_t = fty(Y, t, nu, delta)
        return ((S*d1_t - K*math.exp(-r*Y)*d2_t) * fty_t)
    return integrate.quad(expr, 0, np.inf)[0]

In [7]:
def value_at_risk(data, alpha = 0.95):
    sorted_data = np.sort(data)
    VaR = np.quantile(sorted_data, alpha)
    return VaR

In [8]:
def monte_carlo_VaR(n, nu, delta, mu, S0, delta_t, teta, sigma, Y, K, r):
    mu_noise, sigma_noise = 0, 1
    white_noise_S = np.random.normal(mu_noise, sigma_noise, size = n)
    
    p_t = option_price(Y, K, S0, d1_cdf, d2_cdf, r, nu, delta, sigma)
    
    t_t = invgamma.rvs(scale=((delta*delta)/2), size=n, a=(nu/2))
    
    S_t_next = np.zeros(len(white_noise_S))
    for i in range(len(white_noise_S)):
        S_t_next[i] = S0 + mu*S0*delta_t + S0*(teta + ((sigma**2)/2))*t_t[i] + sigma*math.sqrt(t_t[i])*white_noise_S[i]
        
    p_t_next = np.zeros(len(white_noise_S))
    for i in range(len(white_noise_S)):
        p_t_next[i] = option_price((Y - delta_t), K, S_t_next[i], d1_cdf, d2_cdf, r, nu, delta, sigma)
    
    p_t_diff = np.zeros(len(white_noise_S))
    for i in range(len(white_noise_S)):
        p_t_diff[i] = p_t_next[i] - p_t
    
    VaR = value_at_risk(p_t_diff)

    return VaR

In [9]:
data_raw = pd.read_csv("data_ALL/nasdaq_TSLA_data.csv")

option_table = pd.pivot_table(data = data_raw, values = 'Last', index = 'Strike', columns = 'Date', fill_value = '-')
option_table = option_table.loc[(option_table.index > 192.5) & (option_table.index < 215)]
var_table = option_table.copy()
var_table.columns.name = None
var_table = var_table.reset_index()
var_table

,Strike,10.02.2023,11.02.2023,12.02.2023,13.02.2023,14.02.2023,15.02.2023,16.02.2023,17.02.2023
0,195.0,16.56,16.51,11.52,8.9,6.55,15.59,19.5,22.1
1,197.5,15.0,14.0,9.23,7.57,5.4,13.54,17.25,19.0
2,200.0,13.45,11.73,8.37,6.52,4.43,11.65,15.1,17.09
3,202.5,11.85,9.98,7.52,5.5,3.6,9.82,12.9,15.49
4,205.0,10.59,8.25,6.5,4.75,2.92,8.25,10.8,13.64
5,207.5,9.35,7.11,5.0,3.98,2.36,6.81,8.95,11.21
6,210.0,8.25,6.21,4.44,3.35,1.92,5.56,7.2,9.99
7,212.5,7.2,5.09,3.75,2.83,1.51,4.5,5.75,8.56


In [10]:
days_count = len(var_table.axes[1]) - 1
strike_count = len(var_table['Strike'])
Y = (days_count + 1)/252

for j in range(strike_count):
    delta_t = 0
    # print("strike = " + str(var_table['Strike'][j]))
    for i in range(days_count):
        delta_t = delta_t + 1/252
        # print("delta_t" + str(delta_t))
        VaR = monte_carlo_VaR(n, nu, delta, mu, S0, delta_t, teta, sigma, Y, var_table['Strike'][j], r)
        # print("p_(t+1)" + str(mc['p_(t+1)'][0]))
        var_table.at[j, var_table.columns[i + 1]] = VaR
    
var_table

,Strike,10.02.2023,11.02.2023,12.02.2023,13.02.2023,14.02.2023,15.02.2023,16.02.2023,17.02.2023
0,195.0,20.398215,17.439923,16.980837,14.644912,12.582984,9.458606,6.500048,1.738269
1,197.5,19.922578,18.401621,16.14181,14.690004,10.856151,9.873068,5.865228,2.208255
2,200.0,21.11954,17.483676,16.415553,14.256165,11.720501,8.122214,6.023338,2.186244
3,202.5,19.849393,19.385506,16.242464,14.16505,11.763062,9.301298,5.474278,2.636403
4,205.0,19.570994,18.558282,15.347842,13.874757,11.38161,8.272163,6.328363,2.294144
5,207.5,20.460475,17.629859,16.390173,14.665805,11.181655,8.697775,5.26849,1.842774
6,210.0,20.133661,17.222634,15.913366,12.971725,11.349459,8.864418,6.179311,1.66549
7,212.5,20.221994,17.327457,14.998377,13.275748,10.79803,8.193771,6.356446,2.073654


In [26]:
option_table.columns.name = None
option_table = option_table.reset_index()
failures_table = option_table.copy()

for j in range(strike_count):
    p0 = option_price(Y, var_table['Strike'][j], S0, d1_cdf, d2_cdf, r, nu, delta, sigma)
    for i in range(days_count):
        VaR = var_table.at[j, var_table.columns[i + 1]]
        p_diff = option_table.at[j, option_table.columns[i + 1]] - p0
        if (abs(p_diff) > abs(VaR)):
            failures_table.at[j, failures_table.columns[i + 1]] = 0
        else:
            failures_table.at[j, failures_table.columns[i + 1]] = 1

failures_table

,Strike,10.02.2023,11.02.2023,12.02.2023,13.02.2023,14.02.2023,15.02.2023,16.02.2023,17.02.2023
0,195.0,1,1,1,1,1,1,0,0
1,197.5,1,1,1,1,1,1,0,0
2,200.0,1,1,1,1,1,1,0,0
3,202.5,1,1,1,1,1,1,0,0
4,205.0,1,1,1,1,1,1,1,0
5,207.5,1,1,1,1,1,1,1,0
6,210.0,1,1,1,1,1,1,1,0
7,212.5,1,1,1,1,1,1,1,0


In [69]:
prob = 0.05
n = days_count*strike_count
x = 0
for j in range(strike_count):
    for i in range(days_count):
        if failures_table.at[j, failures_table.columns[i + 1]] == 0:
            x += 1
LR = -2*math.log(((1 - prob)**(n - x))*(prob**x)) + 2*math.log(((1 - (x/n))**(n - x))*((x/n)**x))
print("Test statistics = " + str(LR))

p_value = 1 - chi2.cdf(LR, 1)
print("p-value = " + str(p_value))

Test statistics = 15.462148838943499
p-value = 8.417412780192812e-05


In [70]:
prob_arr = [0.05, 0.1, 0.11, 0.12, 0.15, 0.2]
test_table = pd.DataFrame({'Null hypothesis probability': prob_arr, 'LR': [0, 0, 0, 0, 0, 0], 'p-value': [0, 0, 0, 0, 0, 0]})

for i in range(len(prob_arr)):
    LR = -2*math.log(((1 - prob_arr[i])**(n - x))*(prob_arr[i]**x)) + 2*math.log(((1 - (x/n))**(n - x))*((x/n)**x))
    p_value = 1 - chi2.cdf(LR, 1)
    
    test_table.loc[test_table['Null hypothesis probability'] == prob_arr[i], ['LR']] = LR
    test_table.loc[test_table['Null hypothesis probability'] == prob_arr[i], ['p-value']] = p_value

test_table

,Null hypothesis probability,LR,p-value
0,0.05,15.462149,0.000084
1,0.10,4.449608,0.034909
2,0.11,3.324186,0.068268
3,0.12,2.411067,0.120480
4,0.15,0.662920,0.415531
5,0.20,0.063511,0.801030
